In [ ]:
import sys
import copy
import matplotlib.pyplot as plt
sys.path.append("../gotmtool")
from gotmtool import *

## Create a model

In [ ]:
m = Model(name='BGC_MEMG', environ='../gotmtool/.gotm_env.yaml')

In [ ]:
for key in m.environ:
    print('{:>15s}: {}'.format(key, m.environ[key]) )

## Build the model

In [ ]:
%%time
m.build(use_fabm=True)

## Configuration
Initialize the GOTM configuration from a yaml file

In [ ]:
cfg = config_load('gotm.yaml')

Modify the path for input files

In [ ]:
datadir = os.path.join(os.getcwd(), 'forcing')

profile_data = {
    'temperature': 'TEMP_profiles.dat',
    'salinity':    'PSAL_profiles.dat',
}

surface_data = {
    'u10':    'Uwind.dat',
    'v10':    'Vwind.dat',
    'airp':   'Pair.dat',
    'airt':   'Tair.dat',
    'hum':    'Qair.dat',
    'cloud':  'cloud.dat',
    'precip': 'rain.dat',
    'swr':    'swrad.dat',
    'longwave_radiation': 'lwrad.dat',
    }

fabm_profile_data = {
    '2p2z_no3':     'NITRATE_profiles.dat',
    '2p2z_phyS':    'Phyt_profiles.dat',
    '2p2z_chlS':    'CHLA_profiles.dat',
    '2p2z_LDeN':    'largePOC_profiles.dat',
    '2p2z_pre_no3': 'NITRATE_profiles.dat',
    }

cfg['version'] = 6
for key in profile_data:
    cfg[key]['method'] = 'file'
    cfg[key]['file'] = os.path.join(datadir, profile_data[key])
    
for key in surface_data:
    cfg['surface'][key]['method'] = 'file'
    cfg['surface'][key]['file'] = os.path.join(datadir, surface_data[key])

for key in fabm_profile_data:
    cfg['fabm']['input'][key]['method'] = 'file'
    cfg['fabm']['input'][key]['file'] = os.path.join(datadir, fabm_profile_data[key])

Initialize FABM configuration from a yaml file

In [ ]:
cfg_fabm = config_load('fabm.yaml')

## Run the model

In [ ]:
%%time
sim = m.run(config=cfg, label='test01', config_fabm=cfg_fabm)

## Take a look at the results

We can quickly take a look at the simulation results from the returned `Simulation` object.

In [ ]:
ds = sim.load_data()
ds

In [ ]:
varnames = ['temp', 'salt', '2p2z_phyS', '2p2z_zooS']
nv = len(varnames)
fig, axarr = plt.subplots(nv, sharex='col')
fig.set_size_inches([10, 12])
for i, varname in enumerate(varnames):
    ds.data_vars[varname].plot(ax=axarr[i])
    axarr[i].set_title('')
    axarr[i].set_xlabel('')
    axarr[i].set_ylim([-1000, 0])

## Run the model with different turbulence closure methods in a batch

We can also run a set of simulations with slightly different configurations (either different `gotm.yaml` or different `fabm.yaml` but not both) in a batch, which can take advantage of multiple processors on the server. Here we domenstrate this by running this case using three turbulence closure schemes:
- The $k$-$\epsilon$ model with the [Schumann and Gerz (1995)](https://doi.org/10.1175/1520-0450-34.1.33) stability function, the one used in [Wang and Fennel, 2023](https://doi.org/10.1029/2022GL101220).
- The generic length scale (GLS; [Umlauf and Burchard, 2003](https://doi.org/10.1357/002224003322005087)) model in the $k$-$\epsilon$ formulation with the weak-equilibrium stability function by [Canuto et al., 2001](https://doi.org/10.1175/1520-0485(2001)031%3C1413:OTPIOP%3E2.0.CO;2) (C01A).
- The K-profile parameterization (KPP; [Large et al., 1994](https://doi.org/10.1029/94RG01872), [Griffies et al., 2015](https://github.com/CVMix/CVMix-description/raw/master/cvmix.pdf)) with [CVMix](http://cvmix.github.io).

In [ ]:
# collect the configurations and the labels of the three runs
cfgs = []
labels = []

In [ ]:
cfgs.append(copy.deepcopy(cfg))
labels.append('keps-SG95')

In [ ]:
cfg['turbulence']['turb_method'] = 'second_order'
cfg['turbulence']['tke_method'] = 'tke'
cfg['turbulence']['len_scale_method'] = 'gls'
cfg['turbulence']['scnd']['method'] =  'weak_eq_kb_eq'
cfg['turbulence']['scnd']['scnd_coeff'] =  'canuto-a'
cfg['turbulence']['turb_param']['length_lim'] = 'false'
cfg['turbulence']['turb_param']['compute_c3'] = 'true'
cfg['turbulence']['turb_param']['Ri_st'] = 0.25
cfg['turbulence']['generic']['gen_m'] = 1.5 
cfg['turbulence']['generic']['gen_n'] = -1.0
cfg['turbulence']['generic']['gen_p'] = 3.0 
cfg['turbulence']['generic']['cpsi1'] = 1.44
cfg['turbulence']['generic']['cpsi2'] = 1.92
cfg['turbulence']['generic']['cpsi3minus'] = -0.63
cfg['turbulence']['generic']['cpsi3plus'] = 1.0 
cfg['turbulence']['generic']['sig_kpsi'] = 1.0 
cfg['turbulence']['generic']['sig_psi'] = 1.3
cfgs.append(copy.deepcopy(cfg))
labels.append('GLS-C01A')

In [ ]:
cfg['turbulence']['turb_method'] = 'cvmix'
cfg['cvmix']['surface_layer']['kpp']['langmuir_method'] = 'none'
cfgs.append(copy.deepcopy(cfg))
labels.append('KPP-CVMix')

Run the three runs with the same `fabm.yaml` using 3 processers. 

In [ ]:
%%time
sims = m.run_batch(configs=cfgs, labels=labels, configs_fabm=[cfg_fabm], nproc=3)

This returns a list of `Simulation` objects.

In [ ]:
sims